In [1]:
import pandas as pd
import numpy as np

In [65]:
import h5py as h5
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Embedding, Dropout, Activation, Reshape
#from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
#Over-sampling: SMOTE
#SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
#based on those that already exist. It works randomly picking a point from the minority class and computing 
#the k-nearest neighbors for this point.The synthetic points are added between the chosen point and its neighbors.
#We'll use ratio='minority' to resample the minority class.
smote = SMOTE('minority')

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


In [ ]:
#dataset = pd.read_csv('sample_data/data_CKD_MI.csv')

In [76]:
dataset = pd.read_csv('sample_data/data_CKD_bigdata_refined.csv')


In [77]:
dataset.head()

,sex,age,sido,height,weight,waist,vision_left,vision_right,listen_left,listen_right,BP_HIGH,BP_LWST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE_CD,CREATININE,SGOT_AST,SGPT_ALT,GAMMA_GTP,SMK_STAT_TYPE_RSPS_CD,DRINK_OR_NOT,MOUTH_CHECK,BMI,GFR,Class
0,1,8,43,170,75,90.0,1.0,1.0,1.0,1.0,120.0,80.0,99.0,193.0,92.0,48.0,126.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,1.0,1,25.951557,86.886038,2
1,1,7,11,180,80,89.0,0.9,1.2,1.0,1.0,130.0,82.0,106.0,228.0,121.0,55.0,148.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,0.0,1,24.691358,100.644197,1
2,1,9,41,165,75,91.0,1.2,1.5,1.0,1.0,120.0,70.0,98.0,136.0,104.0,41.0,74.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,0.0,0,27.548209,95.928640,1
3,1,11,48,175,80,91.0,1.5,1.2,1.0,1.0,145.0,87.0,95.0,201.0,106.0,76.0,104.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,0.0,1,26.122449,73.203730,2
4,1,11,30,165,60,80.0,1.0,1.2,1.0,1.0,138.0,82.0,101.0,199.0,104.0,61.0,117.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,0.0,1,22.038567,105.714492,1


In [78]:
dataset.shape

(1000000, 29)

In [79]:
dataset.isnull().sum()

sex                      0
age                      0
sido                     0
height                   0
weight                   0
waist                    0
vision_left              0
vision_right             0
listen_left              0
listen_right             0
BP_HIGH                  0
BP_LWST                  0
BLDS                     0
TOT_CHOLE                0
TRIGLYCERIDE             0
HDL_CHOLE                0
LDL_CHOLE                0
HMG                      0
OLIG_PROTE_CD            0
CREATININE               0
SGOT_AST                 0
SGPT_ALT                 0
GAMMA_GTP                0
SMK_STAT_TYPE_RSPS_CD    0
DRINK_OR_NOT             0
MOUTH_CHECK              0
BMI                      0
GFR                      0
Class                    0
dtype: int64

In [ ]:
#dataset.iloc[:,:-1]

In [ ]:
#dataset.iloc[:,-1]

In [80]:
# Data preprocessing
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [24]:
X

array([[  1.        ,   8.        ,  43.        , ...,   1.        ,
         25.95155709,  86.88603751],
       [  1.        ,   7.        ,  11.        , ...,   1.        ,
         24.69135802, 100.6441974 ],
       [  1.        ,   9.        ,  41.        , ...,   0.        ,
         27.54820937,  95.92863959],
       ...,
       [  2.        ,   9.        ,  26.        , ...,   0.        ,
         20.81165453,  81.54215818],
       [  1.        ,   6.        ,  29.        , ...,   0.        ,
         19.59183673, 118.6956429 ],
       [  1.        ,  11.        ,  41.        , ...,   0.        ,
         27.34375   ,  92.27934768]])

In [25]:
y

array([2, 1, 1, ..., 2, 1, 1])

In [ ]:
# from sklearn.model_selection import train_test_split    
# from sklearn.preprocessing import MinMaxScaler
# from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
# import keras.layers
# from tensorflow.keras.utils import to_categorical
# from imblearn.over_sampling import ADASYN
# from imblearn.combine import SMOTEENN

# # Data preprocessing
# X = dataset.iloc[:,:-1].values
# y = dataset.iloc[:,-1].values
# # print(X.shape)
# # print(y.shape)
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2)
# X_sm, y_sm = smote.fit_resample(X, y)

# from sklearn.utils import class_weight
# class_weight = class_weight.compute_class_weight('balanced',y)
# # scaler = MinMaxScaler()
# # X_train = scaler.fit_transform(X_train)
# # X_test  = scaler.transform(X_test)
# # print(X_train.shape,y_train.shape)
# # y_train = to_categorical(y_train,dtype="uint8")
# # print(y_train.shape)
# # y_test = to_categorical(y_test,dtype ="uint8")
# # print(y_test.shape)

# # print(X_train.shape)
# # print(X_test.shape)

In [ ]:
# from sklearn.utils import class_weight
# classes_weights = list(class_weight.compute_class_weight('balanced',
#                                              np.unique(dataset['Class']),
#                                              dataset['Class']))


In [83]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(class_weight='balanced',y=dataset['Class'])

In [84]:
classes_weights

array([0.41136689, 0.42071075, 0.42071075, ..., 0.41136689, 0.42071075,
       0.42071075])

In [93]:
from xgboost import XGBClassifier

In [94]:
xgb= XGBClassifier(n_estimators=1000)

In [ ]:
xgb.fit(X, y, sample_weight=classes_weights)

XGBClassifier(n_estimators=1000, objective='multi:softprob')

In [36]:
from collections import Counter

In [37]:
counter = Counter(y)
print('Before',counter)

Before Counter({2: 486184, 1: 475386, 3: 36377, 5: 1027, 4: 1026})


In [85]:
len(classes_weights)

1000000

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2)

In [9]:
print(class_weight)
# (unique,counts)=np.unique(y_train,return_counts=True())
(unique,counts) = np.unique(y_sm,return_counts=True)
print(unique, counts)

<module 'sklearn.utils.class_weight' from '/usr/local/lib/python3.7/dist-packages/sklearn/utils/class_weight.py'>
[1 2 3 4 5] [475386 486184  36377 486184   1027]


In [88]:
from tensorflow.keras.utils import to_categorical as cat

In [89]:
#X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state = 2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
print(X_train.shape,y_train.shape)
y_train = cat(y_train, num_classes = 6 )
print(y_train.shape)
y_test = cat(y_test,num_classes=6)
print(y_test.shape)

(800000, 28) (800000, 6)
(800000, 6, 6)
(200000, 6, 6)


In [14]:
(unique,counts) = np.unique(y_train,return_counts=True)
print(unique, counts)

[0. 1.] [5940630 1188126]


In [ ]:
#X_train,y_train = smote.fit_resample(X_train,y_train)
#print(X_train.shape,y_train.shape)

In [44]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

print(class_weight)
class_weight = np.insert(class_weight,5,0)
print(class_weight)

(800000, 28, 1)
(200000, 28, 1)
(800000,)
(200000,)
<module 'sklearn.utils.class_weight' from '/usr/local/lib/python3.7/dist-packages/sklearn/utils/class_weight.py'>


IndexError: ignored

In [90]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [91]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def evaluate_model_CNN(X_train, X_val, y_train, y_val):
  model = LSTM
  timesteps=125
  dims=4
  model=Sequential()
#   model.add(LSTM(24,
#   dropout=0.2,return_sequences=True,
#   activation='tanh', recurrent_activation='tanh',recurrent_dropout=0.2,input_shape=(1,X_train.shape[2])))
#   model.add(LSTM(12, 
#   activation='tanh', recurrent_activation='tanh',
#   dropout=0.2,recurrent_dropout=0.2))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(keras.layers.Dense (64,activation='relu')) #input layer
  model.add(Dropout(0.2))
  model.add(keras.layers.Dense (32,activation='relu')) #hidden
  model.add(Dropout(0.3))
  model.add(keras.layers.Dense (16,activation='relu')) #hidden
  model.add(Dropout(0.3))
  model.add(keras.layers.Dense (6,activation='softmax')) #output


  epochs = 400
  batch_size = 64
  model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
  history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=4, batch_size=128, verbose=1,class_weight=class_weight)
  #val_acc = model.evaluate(X_val, y_val, verbose = 1)
  return history, val_acc

#model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())

In [92]:
n_folds = 1
cv_scores, model_history = list(), list()
for _ in range(n_folds):  
  # split data
  # x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
  # evaluate model
  print(X_train.shape)
  print(y_train.shape)
  print(X_test.shape)
  print(y_test.shape)
  model = tf.keras.Sequential()
  history, test_acc = evaluate_model_CNN(X_train, X_test, y_train, y_test)
# list all data in history
#print(history.history.keys())
# summarize history for accuracy
#plt.plot(history.history['accuracy'])

(800000, 28, 1)
(800000, 6, 6)
(200000, 28, 1)
(200000, 6, 6)


AttributeError: ignored

In [ ]:
y_prob = model.predict(X_test)
print(y_prob)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
y_classes_1=y_test.argmax(axis=-1)
mat = confusion_matrix(y_classes_1,y_classes)
print(mat)
pl= ConfusionMatrixDisplay(mat,display_labels=range(5))

In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_classes_1,y_classes, target_names=['Stage 1', 'Stage 2', 'Stage 3','Stage 4','Stage 5']))


In [ ]:
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D

# def evaluate_model_LSTM(X_train, X_val, y_train, y_val):
#   model = LSTM
#   timesteps=125
#   dims=4
#   model=Sequential()
#   model.add(LSTM(24,
#   dropout=0.2,return_sequences=True,
#   activation='tanh', recurrent_activation='tanh',recurrent_dropout=0.2,input_shape=(X_train.shape[1],1)))
#   model.add(LSTM(12, 
#   activation='tanh', recurrent_activation='tanh',
#   dropout=0.2,recurrent_dropout=0.2))
#   model.add(keras.layers.Dense (64,activation='relu')) #input layer
#   model.add(Dropout(0.2))
#   model.add(keras.layers.Dense (32,activation='relu')) #hidden
#   model.add(Dropout(0.3))
#   model.add(keras.layers.Dense (16,activation='relu')) #hidden
#   model.add(Dropout(0.3))
#   model.add(keras.layers.Dense (5,activation='softmax')) #output


#   epochs = 400
#   batch_size = 64
#   model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
#   model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=100, batch_size=128, verbose=1)
#   val_acc = model.evaluate(X_val, y_val, verbose = 1)
#   return model, val_acc

# #model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
# # model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(MaxPooling1D(pool_size=2))
# # model.add(Flatten())

In [ ]:
# n_folds = 1
# cv_scores, model_history = list(), list()
# for _ in range(n_folds):  
#   # split data
#   # x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
#   # evaluate model
#   print(X_train.shape)
#   print(y_train.shape)
#   print(X_test.shape)
#   print(y_test.shape)
#   model = tf.keras.Sequential()
#   model, test_acc = evaluate_model_LSTM(X_train, X_test, y_train, y_test)
# # list all data in history
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['accuracy'])